In [1]:
%pip install mediapipe opencv-python --user
%pip install opencv-python-headless
%pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install mediapipe opencv-python

In [3]:
!pip install opencv-python mediapipe numpy scikit-learn matplotlib tk pillow roboflow tensorflow

  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl (54.8 MB)
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.10.0
error: uninstall-no-record-file

× Cannot uninstall opencv-python-headless 4.10.0
╰─> The package's contents are unknown: no RECORD file was found for opencv-python-headless.

hint: The package was installed by conda. You should check if it can uninstall the package.


In [4]:
!pip install tensorflow

In [5]:
!pip install roboflow
!pip install roboflow --upgrade
!pip install --ignore-installed roboflow

  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl (54.8 MB)
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.10.0
error: uninstall-no-record-file

× Cannot uninstall opencv-python-headless 4.10.0
╰─> The package's contents are unknown: no RECORD file was found for opencv-python-headless.

hint: The package was installed by conda. You should check if it can uninstall the package.
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl (54.8 MB)
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.10.0
error: uninstall-no-record-file

× Cannot uninstall opencv-python-headless 4.10.0
╰─> The package's contents are unknown: no RECORD file was f

In [8]:
import cv2
import numpy as np
import mediapipe as mp
import os
import time
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
from PIL import Image, ImageTk
import threading
import tensorflow as tf
from roboflow import Roboflow
import json


class SpanishDanceAnalyzer:
    """Main class to analyze Spanish dance performances using pose estimation"""
    
    def __init__(self, use_roboflow=True, roboflow_api_key=None):
        # MediaPipe setup
        self.mp_pose = mp.solutions.pose
        self.mp_drawing = mp.solutions.drawing_utils
        self.pose = self.mp_pose.Pose(
            static_image_mode=False,
            model_complexity=2,
            enable_segmentation=False,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        
        # Roboflow setup for custom Spanish dance position detection
        self.use_roboflow = use_roboflow
        if use_roboflow and roboflow_api_key:
            try:
                rf = Roboflow(api_key=roboflow_api_key)
                # Update with your actual workspace and project name
                self.project = rf.workspace("your-workspace").project("spanish-dance-positions")
                self.model = self.project.version(1).model
                print("Roboflow model loaded successfully")
            except Exception as e:
                print(f"Failed to load Roboflow model: {e}")
                self.use_roboflow = False
        
        # Define key landmarks for Spanish dance analysis
        self.landmarks = {
            # Upper body landmarks (for arm positions)
            'left_wrist': self.mp_pose.PoseLandmark.LEFT_WRIST.value,
            'right_wrist': self.mp_pose.PoseLandmark.RIGHT_WRIST.value,
            'left_elbow': self.mp_pose.PoseLandmark.LEFT_ELBOW.value,
            'right_elbow': self.mp_pose.PoseLandmark.RIGHT_ELBOW.value,
            'left_shoulder': self.mp_pose.PoseLandmark.LEFT_SHOULDER.value,
            'right_shoulder': self.mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
            
            # Lower body landmarks (for footwork)
            'left_hip': self.mp_pose.PoseLandmark.LEFT_HIP.value,
            'right_hip': self.mp_pose.PoseLandmark.RIGHT_HIP.value,
            'left_knee': self.mp_pose.PoseLandmark.LEFT_KNEE.value,
            'right_knee': self.mp_pose.PoseLandmark.RIGHT_KNEE.value,
            'left_ankle': self.mp_pose.PoseLandmark.LEFT_ANKLE.value,
            'right_ankle': self.mp_pose.PoseLandmark.RIGHT_ANKLE.value,
            'left_foot_index': self.mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value,
            'right_foot_index': self.mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value,
        }
        
        # Landmark groups for specific analysis
        self.arm_landmarks = [
            'left_wrist', 'right_wrist', 'left_elbow', 
            'right_elbow', 'left_shoulder', 'right_shoulder'
        ]
        
        self.foot_landmarks = [
            'left_hip', 'right_hip', 'left_knee', 'right_knee',
            'left_ankle', 'right_ankle', 'left_foot_index', 'right_foot_index'
        ]
        
        # Storage for reference poses and choreography sequence
        self.reference_poses = {}
        self.choreography_sequence = []
        
        # Data directory for saving/loading
        self.data_dir = "spanish_dance_data"
        os.makedirs(self.data_dir, exist_ok=True)
    
    def extract_poses_from_video(self, video_path, skip_frames=2):
        """Process video and extract pose landmarks for each frame"""
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Could not open video: {video_path}")
            return [], [], None
            
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = frame_count / fps if fps > 0 else 0
        
        print(f"Processing video: {os.path.basename(video_path)}")
        print(f"Duration: {duration:.2f} seconds, Total frames: {frame_count}")
        
        poses = []
        frames = []
        keyframes = []  # Store key frames for choreography points
        processed_count = 0
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            processed_count += 1
            if processed_count % skip_frames != 0:
                continue
                
            # Convert to RGB for MediaPipe
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Process with MediaPipe
            results = self.pose.process(rgb_frame)
            
            if results.pose_landmarks:
                # Store frame and pose data
                frames.append(frame)
                
                # Extract key landmarks
                landmarks_dict = {}
                for name, idx in self.landmarks.items():
                    lm = results.pose_landmarks.landmark[idx]
                    landmarks_dict[name] = {
                        'x': lm.x, 'y': lm.y, 'z': lm.z, 'visibility': lm.visibility
                    }
                
                pose_data = {
                    'frame_idx': processed_count,
                    'timestamp': processed_count / fps if fps > 0 else 0,
                    'landmarks': landmarks_dict,
                    'full_pose': results.pose_landmarks
                }
                
                poses.append(pose_data)
                
                # Every 30 frames (adjusted by skip_frames), save a keyframe for reference
                if processed_count % (30 * skip_frames) == 0:
                    # Draw landmarks on frame for visualization
                    annotated_frame = frame.copy()
                    self.mp_drawing.draw_landmarks(
                        annotated_frame, 
                        results.pose_landmarks,
                        self.mp_pose.POSE_CONNECTIONS
                    )
                    keyframes.append({
                        'frame_idx': processed_count,
                        'timestamp': processed_count / fps if fps > 0 else 0,
                        'frame': annotated_frame
                    })
            
            # Show progress
            if processed_count % 100 == 0:
                print(f"Processed {processed_count} frames...")
        
        cap.release()
        print(f"Video processing complete. Extracted {len(poses)} poses.")
        return poses, frames, keyframes
    
    def normalize_pose_data(self, landmarks_dict):
        """Normalize pose data to make it scale and position invariant"""
        if not landmarks_dict:
            return None
            
        # Extract hip center coordinates
        left_hip = np.array([
            landmarks_dict['left_hip']['x'],
            landmarks_dict['left_hip']['y'],
            landmarks_dict['left_hip']['z']
        ])
        
        right_hip = np.array([
            landmarks_dict['right_hip']['x'],
            landmarks_dict['right_hip']['y'],
            landmarks_dict['right_hip']['z']
        ])
        
        hip_center = (left_hip + right_hip) / 2
        
        # Calculate torso height for normalization
        left_shoulder = np.array([
            landmarks_dict['left_shoulder']['x'],
            landmarks_dict['left_shoulder']['y'],
            landmarks_dict['left_shoulder']['z']
        ])
        
        right_shoulder = np.array([
            landmarks_dict['right_shoulder']['x'],
            landmarks_dict['right_shoulder']['y'],
            landmarks_dict['right_shoulder']['z']
        ])
        
        shoulder_center = (left_shoulder + right_shoulder) / 2
        torso_height = np.linalg.norm(shoulder_center - hip_center)
        
        # Normalize landmarks relative to hip center and torso height
        normalized_landmarks = {}
        for name, lm in landmarks_dict.items():
            point = np.array([lm['x'], lm['y'], lm['z']])
            # Translate to make hip center the origin
            normalized = point - hip_center
            # Scale by torso height
            if torso_height > 0:
                normalized = normalized / torso_height
            
            normalized_landmarks[name] = {
                'x': normalized[0],
                'y': normalized[1],
                'z': normalized[2],
                'visibility': lm['visibility']
            }
        
        return normalized_landmarks
    
    def normalize_all_poses(self, poses):
        """Normalize an array of poses"""
        normalized_poses = []
        for pose in poses:
            norm_landmarks = self.normalize_pose_data(pose['landmarks'])
            if norm_landmarks:
                normalized_pose = pose.copy()
                normalized_pose['normalized_landmarks'] = norm_landmarks
                normalized_poses.append(normalized_pose)
        
        return normalized_poses
    
    def set_reference_demo(self, video_path):
        """Set the demonstration video as reference for analysis"""
        poses, frames, keyframes = self.extract_poses_from_video(video_path)
        self.reference_poses = self.normalize_all_poses(poses)
        self.reference_keyframes = keyframes
        
        # Save reference data
        self.save_reference_data()
        
        return len(self.reference_poses)
    
    def save_reference_data(self):
        """Save reference pose data to disk"""
        # We don't save full frames to save disk space
        serializable_poses = []
        for pose in self.reference_poses:
            pose_copy = pose.copy()
            # Remove non-serializable objects
            if 'full_pose' in pose_copy:
                del pose_copy['full_pose']
            serializable_poses.append(pose_copy)
        
        with open(os.path.join(self.data_dir, "reference_poses.pkl"), "wb") as f:
            pickle.dump(serializable_poses, f)
        
        # Save keyframes as images
        keyframes_dir = os.path.join(self.data_dir, "keyframes")
        os.makedirs(keyframes_dir, exist_ok=True)
        
        keyframe_data = []
        for i, kf in enumerate(self.reference_keyframes):
            filename = f"keyframe_{i:03d}.jpg"
            filepath = os.path.join(keyframes_dir, filename)
            cv2.imwrite(filepath, kf['frame'])
            
            keyframe_data.append({
                'frame_idx': kf['frame_idx'],
                'timestamp': kf['timestamp'],
                'filename': filename
            })
        
        with open(os.path.join(self.data_dir, "keyframes.json"), "w") as f:
            json.dump(keyframe_data, f)
    
    def load_reference_data(self):
        """Load reference pose data from disk"""
        ref_file = os.path.join(self.data_dir, "reference_poses.pkl")
        if os.path.exists(ref_file):
            with open(ref_file, "rb") as f:
                self.reference_poses = pickle.load(f)
            
            # Load keyframe data
            keyframes_data_file = os.path.join(self.data_dir, "keyframes.json")
            keyframes_dir = os.path.join(self.data_dir, "keyframes")
            
            if os.path.exists(keyframes_data_file):
                with open(keyframes_data_file, "r") as f:
                    keyframe_data = json.load(f)
                
                self.reference_keyframes = []
                for kf in keyframe_data:
                    filepath = os.path.join(keyframes_dir, kf['filename'])
                    if os.path.exists(filepath):
                        frame = cv2.imread(filepath)
                        self.reference_keyframes.append({
                            'frame_idx': kf['frame_idx'],
                            'timestamp': kf['timestamp'],
                            'frame': frame
                        })
            
            return True
        return False
    
    def define_choreography_sequence(self, timestamps, labels):
        """Define key moments in the choreography with labels"""
        self.choreography_sequence = []
        
        for timestamp, label in zip(timestamps, labels):
            # Find the closest pose to the timestamp
            closest_pose = None
            min_diff = float('inf')
            
            for pose in self.reference_poses:
                diff = abs(pose['timestamp'] - timestamp)
                if diff < min_diff:
                    min_diff = diff
                    closest_pose = pose
            
            if closest_pose:
                self.choreography_sequence.append({
                    'timestamp': timestamp,
                    'label': label,
                    'pose_idx': closest_pose['frame_idx']
                })
        
        # Save choreography sequence
        with open(os.path.join(self.data_dir, "choreography.json"), "w") as f:
            json.dump(self.choreography_sequence, f)
        
        return len(self.choreography_sequence)
    
    def load_choreography_sequence(self):
        """Load choreography sequence from disk"""
        choreo_file = os.path.join(self.data_dir, "choreography.json")
        if os.path.exists(choreo_file):
            with open(choreo_file, "r") as f:
                self.choreography_sequence = json.load(f)
            return True
        return False
    
    def extract_features(self, landmarks_dict, landmark_group="all"):
        """Extract pose features for comparison"""
        if landmark_group == "arms":
            target_landmarks = self.arm_landmarks
        elif landmark_group == "feet":
            target_landmarks = self.foot_landmarks
        else:
            target_landmarks = list(self.landmarks.keys())
        
        features = []
        for name in target_landmarks:
            if name in landmarks_dict:
                lm = landmarks_dict[name]
                features.extend([lm['x'], lm['y'], lm['z']])
        
        return np.array(features)
    
    def compute_pose_similarity(self, pose1, pose2, landmark_group="all"):
        """Compute similarity between two poses"""
        # Extract features from normalized landmarks
        features1 = self.extract_features(pose1['normalized_landmarks'], landmark_group)
        features2 = self.extract_features(pose2['normalized_landmarks'], landmark_group)
        
        # Compute cosine similarity
        similarity = cosine_similarity(features1.reshape(1, -1), features2.reshape(1, -1))[0][0]
        
        # Normalize to percentage (0-100%)
        similarity_percent = (similarity + 1) * 50
        
        return similarity_percent
    
    def find_matching_sequence(self, student_poses, window_size=3):
        """Find the best matching sequence in student video for each choreography point"""
        results = []
        
        for choreo_point in self.choreography_sequence:
            # Find reference pose for this choreography point
            ref_pose_idx = None
            for i, pose in enumerate(self.reference_poses):
                if pose['frame_idx'] == choreo_point['pose_idx']:
                    ref_pose_idx = i
                    break
            
            if ref_pose_idx is None:
                continue
            
            ref_pose = self.reference_poses[ref_pose_idx]
            
            # Find best matching pose in student video
            best_match_idx = -1
            best_similarity = 0
            
            for i, student_pose in enumerate(student_poses):
                # Check if we have enough window space
                if i < len(student_poses) - window_size:
                    # Compute average similarity over a window of poses
                    window_similarity = 0
                    for w in range(window_size):
                        pose_similarity = self.compute_pose_similarity(
                            ref_pose, student_poses[i + w]
                        )
                        window_similarity += pose_similarity
                    
                    window_similarity /= window_size
                    
                    if window_similarity > best_similarity:
                        best_similarity = window_similarity
                        best_match_idx = i
            
            # Determine arm and footwork specific similarity
            arm_similarity = 0
            foot_similarity = 0
            
            if best_match_idx >= 0:
                best_student_pose = student_poses[best_match_idx]
                arm_similarity = self.compute_pose_similarity(ref_pose, best_student_pose, "arms")
                foot_similarity = self.compute_pose_similarity(ref_pose, best_student_pose, "feet")
            
            results.append({
                'choreography_point': choreo_point,
                'label': choreo_point['label'],
                'reference_pose_idx': ref_pose_idx,
                'student_pose_idx': best_match_idx,
                'overall_similarity': best_similarity,
                'arm_similarity': arm_similarity,
                'foot_similarity': foot_similarity
            })
        
        return results
    
    def analyze_student_performance(self, student_video_path):
        """Analyze a student's dance performance compared to reference"""
        # Make sure we have reference data
        if not self.reference_poses:
            if not self.load_reference_data():
                print("Error: No reference data available. Please set a reference demo first.")
                return None
        
        if not self.choreography_sequence:
            if not self.load_choreography_sequence():
                print("Error: No choreography sequence defined. Please define choreography first.")
                return None
        
        # Process student video
        student_poses, _, student_keyframes = self.extract_poses_from_video(student_video_path)
        normalized_student_poses = self.normalize_all_poses(student_poses)
        
        # Find matching sequences
        matching_results = self.find_matching_sequence(normalized_student_poses)
        
        # Calculate overall performance score
        overall_score = 0
        arm_score = 0
        foot_score = 0
        
        if matching_results:
            overall_score = sum(r['overall_similarity'] for r in matching_results) / len(matching_results)
            arm_score = sum(r['arm_similarity'] for r in matching_results) / len(matching_results)
            foot_score = sum(r['foot_similarity'] for r in matching_results) / len(matching_results)
        
        performance_analysis = {
            'student_video': student_video_path,
            'poses_extracted': len(student_poses),
            'matching_results': matching_results,
            'overall_score': overall_score,
            'arm_score': arm_score,
            'foot_score': foot_score
        }
        
        return performance_analysis
    
    def generate_feedback(self, performance_analysis):
        """Generate detailed feedback from performance analysis"""
        if not performance_analysis:
            return "No performance analysis data available."
        
        overall_score = performance_analysis['overall_score']
        arm_score = performance_analysis['arm_score']
        foot_score = performance_analysis['foot_score']
        
        results = performance_analysis['matching_results']
        
        # Generate HTML report
        html = f"""
        <h2>Spanish Dance Performance Evaluation</h2>
        <h3>Overall Technique Mark: {overall_score:.1f}%</h3>
        <div style="margin-bottom: 20px;">
            <div style="display: inline-block; margin-right: 20px;">
                <strong>Arm Movements:</strong> {arm_score:.1f}%
            </div>
            <div style="display: inline-block;">
                <strong>Footwork:</strong> {foot_score:.1f}%
            </div>
        </div>
        
        <h3>Choreography Evaluation</h3>
        <table style="width: 100%; border-collapse: collapse; margin-bottom: 20px;">
            <tr style="background-color: #f2f2f2;">
                <th style="padding: 8px; text-align: left; border: 1px solid #ddd;">Position/Step</th>
                <th style="padding: 8px; text-align: center; border: 1px solid #ddd;">Execution</th>
                <th style="padding: 8px; text-align: center; border: 1px solid #ddd;">Similarity</th>
                <th style="padding: 8px; text-align: left; border: 1px solid #ddd;">Feedback</th>
            </tr>
        """
        
        # Add rows for each choreography point
        for i, result in enumerate(results):
            label = result['label']
            similarity = result['overall_similarity']
            arm_sim = result['arm_similarity']
            foot_sim = result['foot_similarity']
            
            # Determine execution rating
            if similarity >= 90:
                execution = "Excellent"
                color = "green"
                feedback = f"Excellent execution of {label}. Perfect arm position ({arm_sim:.1f}%) and footwork ({foot_sim:.1f}%)."
            elif similarity >= 75:
                execution = "Good"
                color = "#5cb85c"  # lighter green
                feedback = f"Good execution of {label}. "
                if arm_sim < 75:
                    feedback += f"Focus on improving arm positions ({arm_sim:.1f}%). "
                if foot_sim < 75:
                    feedback += f"Pay attention to footwork details ({foot_sim:.1f}%). "
            elif similarity >= 60:
                execution = "Fair"
                color = "orange"
                feedback = f"Fair execution of {label}. "
                if arm_sim < foot_sim:
                    feedback += f"Arm positions need more practice ({arm_sim:.1f}%). "
                else:
                    feedback += f"Footwork needs refinement ({foot_sim:.1f}%). "
            else:
                execution = "Needs Improvement"
                color = "red"
                feedback = f"The {label} position needs significant improvement. Review the demonstration video carefully."
            
            # Add table row
            html += f"""
            <tr>
                <td style="padding: 8px; border: 1px solid #ddd;">{label}</td>
                <td style="padding: 8px; text-align: center; color: {color}; border: 1px solid #ddd;">{execution}</td>
                <td style="padding: 8px; text-align: center; border: 1px solid #ddd;">{similarity:.1f}%</td>
                <td style="padding: 8px; border: 1px solid #ddd;">{feedback}</td>
            </tr>
            """
        
        # Close table and add summary
        html += """
        </table>
        
        <h3>Summary Feedback</h3>
        """
        
        # Generate overall feedback based on scores
        if overall_score >= 85:
            html += """
            <p>Outstanding performance! Your technique closely matches the demonstration video. 
            Continue refining the minor details for even greater precision in your Spanish dance execution.</p>
            """
        elif overall_score >= 70:
            html += """
            <p>Good performance with strong technical foundations. Focus on the positions where your similarity
            score was lower, and pay attention to the precise arm positions and footwork timing.</p>
            """
        elif overall_score >= 50:
            html += """
            <p>Fair performance with room for improvement. Practice the choreography sequence regularly,
            focusing particularly on the positions marked "Needs Improvement" or "Fair".</p>
            """
        else:
            html += """
            <p>This choreography needs more practice. Start by focusing on each position individually
            before attempting the full sequence. Use the demonstration video as a reference and
            practice in front of a mirror.</p>
            """
        
        # Add areas for improvement
        html += "<h3>Priority Areas for Improvement</h3><ul>"
        
        # Sort results by similarity (ascending) to find weakest areas
        sorted_results = sorted(results, key=lambda x: x['overall_similarity'])
        for result in sorted_results[:3]:  # Top 3 weakest areas
            html += f"<li><strong>{result['label']}</strong> ({result['overall_similarity']:.1f}%)</li>"
        
        html += "</ul>"
        
        # Add encouragement
        html += """
        <p style="margin-top: 20px;">
            Remember that consistent practice is key to mastering Spanish dance techniques.
            Focus on one improvement area at a time for the best results.
        </p>
        """
        
        return html
    
    def visualize_comparison(self, performance_analysis, output_path=None):
        """Create a visual comparison of reference and student poses"""
        if not performance_analysis or not performance_analysis['matching_results']:
            return None
        
        results = performance_analysis['matching_results']
        
        # Create a figure with multiple subplots
        rows = len(results)
        fig, axs = plt.subplots(rows, 2, figsize=(12, 4 * rows))
        
        if rows == 1:
            axs = [axs]  # Make it iterable for single row
        
        for i, result in enumerate(results):
            label = result['label']
            similarity = result['overall_similarity']
            
            # Find reference pose
            ref_pose_idx = result['reference_pose_idx']
            ref_pose = self.reference_poses[ref_pose_idx]
            
            # Find the nearest keyframe
            ref_keyframe = None
            min_dist = float('inf')
            for kf in self.reference_keyframes:
                dist = abs(kf['frame_idx'] - ref_pose['frame_idx'])
                if dist < min_dist:
                    min_dist = dist
                    ref_keyframe = kf
            
            if ref_keyframe:
                ref_frame = ref_keyframe['frame']
                
                # Find student pose
                student_pose_idx = result['student_pose_idx']
                if student_pose_idx >= 0:
                    # We would need to have saved student frames, which we don't for memory reasons
                    # Placeholder for actual student frame visualization
                    student_frame = np.zeros_like(ref_frame)
                    
                    # Draw pose comparison
                    axs[i][0].imshow(cv2.cvtColor(ref_frame, cv2.COLOR_BGR2RGB))
                    axs[i][0].set_title(f"Reference: {label}")
                    axs[i][0].axis('off')
                    
                    axs[i][1].imshow(cv2.cvtColor(student_frame, cv2.COLOR_BGR2RGB))
                    axs[i][1].set_title(f"Student: {similarity:.1f}% similarity")
                    axs[i][1].axis('off')
        
        plt.tight_layout()
        
        if output_path:
            plt.savefig(output_path)
            plt.close()
            return output_path
        else:
            plt.show()
            return fig


# GUI Application for the Spanish Dance Analysis Tool
class SpanishDanceApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Spanish Dance Analysis Tool")
        self.root.geometry("1000x700")
        
        # Initialize the analyzer
        self.analyzer = SpanishDanceAnalyzer()
        
        # Try to load existing data
        self.analyzer.load_reference_data()
        self.analyzer.load_choreography_sequence()
        
        # Create the main interface
        self.create_interface()
        
        # Status variables
        self.demo_video_path = None
        self.student_video_path = None
        self.performance_results = None
    
    def create_interface(self):
        """Create the application interface"""
        # Create a notebook with tabs
        notebook = ttk.Notebook(self.root)
        notebook.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Tab 1: Setup and Configuration
        setup_tab = ttk.Frame(notebook)
        notebook.add(setup_tab, text="Setup")
        self.create_setup_tab(setup_tab)
        
        # Tab 2: Student Evaluation
        evaluation_tab = ttk.Frame(notebook)
        notebook.add(evaluation_tab, text="Evaluation")
        self.create_evaluation_tab(evaluation_tab)
        
        # Tab 3: Results and Reports
        results_tab = ttk.Frame(notebook)
        notebook.add(results_tab, text="Results")
        self.create_results_tab(results_tab)
        
        # Status bar
        status_frame = ttk.Frame(self.root)
        status_frame.pack(fill=tk.X, padx=10, pady=5)
        
        self.status_var = tk.StringVar()
        self.status_var.set("Ready")
        status_label = ttk.Label(status_frame, textvariable=self.status_var, relief=tk.SUNKEN, anchor=tk.W)
        status_label.pack(fill=tk.X)
    
    def create_setup_tab(self, parent):
        """Create the setup tab interface"""
        frame = ttk.Frame(parent, padding=10)
        frame.pack(fill=tk.BOTH, expand=True)
        
        # 1. Demo Video Section
        demo_frame = ttk.LabelFrame(frame, text="Step 1: Set Demonstration Video", padding=10)
        demo_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(demo_frame, text="Select a video showing the correct dance choreography:").pack(anchor=tk.W)
        
        btn_frame = ttk.Frame(demo_frame)
        btn_frame.pack(fill=tk.X, pady=5)
        
        ttk.Button(btn_frame, text="Load Demo Video", command=self.load_demo_video).pack(side=tk.LEFT, padx=5)
        
        self.demo_status_var = tk.StringVar()
        self.demo_status_var.set("No demo video loaded")
        ttk.Label(demo_frame, textvariable=self.demo_status_var).pack(anchor=tk.W, pady=5)
        
        # 2. Choreography Definition Section
        choreo_frame = ttk.LabelFrame(frame, text="Step 2: Define Choreography Sequence", padding=10)
        choreo_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(choreo_frame, text="After loading the demo video, define key dance positions/steps:").pack(anchor=tk.W)
        
        choreo_controls = ttk.Frame(choreo_frame)
        choreo_controls.pack(fill=tk.X, pady=5)
        
        self.choreo_list = ttk.Treeview(choreo_frame, columns=("Timestamp", "Label"), show="headings", height=6)
        self.choreo_list.heading("Timestamp", text="Time (seconds)")
        self.choreo_list.heading("Label", text="Position/Step Name")
        self.choreo_list.column("Timestamp", width=100)
        self.choreo_list.column("Label", width=300)
        self.choreo_list.pack(fill=tk.X, pady=5)
        
        choreo_buttons = ttk.Frame(choreo_frame)
        choreo_buttons.pack(fill=tk.X, pady=5)
        
        ttk.Button(choreo_buttons, text="Add Position", command=self.add_choreography_point).pack(side=tk.LEFT, padx=5)
        ttk.Button(choreo_buttons, text="Remove Selected", command=self.remove_choreography_point).pack(side=tk.LEFT, padx=5)
        ttk.Button(choreo_buttons, text="Save Sequence", command=self.save_choreography).pack(side=tk.LEFT, padx=5)
        
        # 3. Configuration Section
        config_frame = ttk.LabelFrame(frame, text="Step 3: Configuration", padding=10)
        config_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(config_frame, text="Add your Roboflow API key (optional):").pack(anchor=tk.W)
        
        api_frame = ttk.Frame(config_frame)
        api_frame.pack(fill=tk.X, pady=5)
        
        self.api_key_var = tk.StringVar()
        api_entry = ttk.Entry(api_frame, textvariable=self.api_key_var, width=40)
        api_entry.pack(side=tk.LEFT, padx=5)
        
        ttk.Button(api_frame, text="Set API Key", command=self.set_api_key).pack(side=tk.LEFT, padx=5)
    
    def create_evaluation_tab(self, parent):
        """Create the evaluation tab interface"""
        frame = ttk.Frame(parent, padding=10)
        frame.pack(fill=tk.BOTH, expand=True)
        
        # 1. Student Video Section
        student_frame = ttk.LabelFrame(frame, text="Step 1: Load Student Performance", padding=10)
        student_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(student_frame, text="Select a video of the student performing the choreography:").pack(anchor=tk.W)
        
        btn_frame = ttk.Frame(student_frame)
        btn_frame.pack(fill=tk.X, pady=5)
        
        ttk.Button(btn_frame, text="Load Student Video", command=self.load_student_video).pack(side=tk.LEFT, padx=5)
        
        self.student_status_var = tk.StringVar()
        self.student_status_var.set("No student video loaded")
        ttk.Label(student_frame, textvariable=self.student_status_var).pack(anchor=tk.W, pady=5)
        
        # 2. Analysis Section
        analysis_frame = ttk.LabelFrame(frame, text="Step 2: Analyze Performance", padding=10)
        analysis_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(analysis_frame, text="Compare the student's performance with the demonstration:").pack(anchor=tk.W)
        
        ttk.Button(analysis_frame, text="Analyze Performance", command=self.analyze_performance).pack(anchor=tk.W, pady=5)
        
        self.analysis_progress = ttk.Progressbar(analysis_frame, orient=tk.HORIZONTAL, mode='indeterminate')
        self.analysis_progress.pack(fill=tk.X, pady=5)
        
        self.analysis_status_var = tk.StringVar()
        self.analysis_status_var.set("Ready for analysis")
        ttk.Label(analysis_frame, textvariable=self.analysis_status_var).pack(anchor=tk.W, pady=5)
        
        # 3. Preview Section
        preview_frame = ttk.LabelFrame(frame, text="Preview", padding=10)
        preview_frame.pack(fill=tk.BOTH, expand=True, pady=5)
        
        preview_container = ttk.Frame(preview_frame)
        preview_container.pack(fill=tk.BOTH, expand=True)
        
        self.demo_preview = ttk.LabelFrame(preview_container, text="Demonstration")
        self.demo_preview.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5)
        
        self.demo_canvas = tk.Canvas(self.demo_preview, bg='black')
        self.demo_canvas.pack(fill=tk.BOTH, expand=True)
        
        self.student_preview = ttk.LabelFrame(preview_container, text="Student Performance")
        self.student_preview.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5)
        
        self.student_canvas = tk.Canvas(self.student_preview, bg='black')
        self.student_canvas.pack(fill=tk.BOTH, expand=True)
    
    def create_results_tab(self, parent):
        """Create the results tab interface"""
        frame = ttk.Frame(parent, padding=10)
        frame.pack(fill=tk.BOTH, expand=True)
        
        # 1. Summary Section
        summary_frame = ttk.LabelFrame(frame, text="Performance Summary", padding=10)
        summary_frame.pack(fill=tk.X, pady=5)
        
        summary_grid = ttk.Frame(summary_frame)
        summary_grid.pack(fill=tk.X, pady=5)
        
        ttk.Label(summary_grid, text="Overall Technique Mark:").grid(row=0, column=0, sticky=tk.W, padx=5, pady=2)
        self.overall_score_var = tk.StringVar()
        ttk.Label(summary_grid, textvariable=self.overall_score_var, font=('TkDefaultFont', 12, 'bold')).grid(row=0, column=1, sticky=tk.W, padx=5, pady=2)
        
        ttk.Label(summary_grid, text="Arm Movements:").grid(row=1, column=0, sticky=tk.W, padx=5, pady=2)
        self.arm_score_var = tk.StringVar()
        ttk.Label(summary_grid, textvariable=self.arm_score_var).grid(row=1, column=1, sticky=tk.W, padx=5, pady=2)
        
        ttk.Label(summary_grid, text="Footwork:").grid(row=2, column=0, sticky=tk.W, padx=5, pady=2)
        self.foot_score_var = tk.StringVar()
        ttk.Label(summary_grid, textvariable=self.foot_score_var).grid(row=2, column=1, sticky=tk.W, padx=5, pady=2)
        
        # 2. Detailed Results Section
        results_frame = ttk.LabelFrame(frame, text="Detailed Results", padding=10)
        results_frame.pack(fill=tk.BOTH, expand=True, pady=5)
        
        # Treeview for results table
        self.results_tree = ttk.Treeview(results_frame, columns=("Position", "Execution", "Similarity", "Feedback"), show="headings", height=10)
        self.results_tree.heading("Position", text="Position/Step")
        self.results_tree.heading("Execution", text="Execution")
        self.results_tree.heading("Similarity", text="Similarity")
        self.results_tree.heading("Feedback", text="Feedback")
        
        self.results_tree.column("Position", width=150)
        self.results_tree.column("Execution", width=100)
        self.results_tree.column("Similarity", width=80)
        self.results_tree.column("Feedback", width=400)
        
        self.results_tree.pack(fill=tk.BOTH, expand=True, pady=5)
        
        # 3. Export Section
        export_frame = ttk.Frame(frame)
        export_frame.pack(fill=tk.X, pady=10)
        
        ttk.Button(export_frame, text="Generate Detailed Report", command=self.generate_report).pack(side=tk.LEFT, padx=5)
        ttk.Button(export_frame, text="Export Results", command=self.export_results).pack(side=tk.LEFT, padx=5)
    
    def load_demo_video(self):
        """Load and process demonstration video"""
        filepath = filedialog.askopenfilename(
            title="Select Demonstration Video",
            filetypes=[("Video files", "*.mp4 *.avi *.mov *.mkv")]
        )
        
        if not filepath:
            return
        
        self.demo_video_path = filepath
        self.demo_status_var.set(f"Processing: {os.path.basename(filepath)}...")
        self.status_var.set("Processing demonstration video. This may take a while...")
        self.root.update()
        
        # Process in a separate thread to not freeze the UI
        def process_video():
            count = self.analyzer.set_reference_demo(filepath)
            
            # Update UI in the main thread
            self.root.after(0, lambda: self.demo_status_var.set(f"Loaded: {os.path.basename(filepath)} - {count} poses extracted"))
            self.root.after(0, lambda: self.status_var.set("Demonstration video processed successfully"))
        
        threading.Thread(target=process_video).start()
    
    def add_choreography_point(self):
        """Add a choreography point to the sequence"""
        if not self.demo_video_path:
            messagebox.showwarning("Warning", "Please load a demonstration video first")
            return
        
        # Simple dialog to get timestamp and label
        dialog = tk.Toplevel(self.root)
        dialog.title("Add Choreography Point")
        dialog.geometry("300x150")
        dialog.resizable(False, False)
        
        ttk.Label(dialog, text="Time (seconds):").pack(anchor=tk.W, padx=10, pady=5)
        time_var = tk.DoubleVar()
        time_entry = ttk.Entry(dialog, textvariable=time_var)
        time_entry.pack(fill=tk.X, padx=10, pady=5)
        
        ttk.Label(dialog, text="Position/Step Name:").pack(anchor=tk.W, padx=10, pady=5)
        label_var = tk.StringVar()
        label_entry = ttk.Entry(dialog, textvariable=label_var)
        label_entry.pack(fill=tk.X, padx=10, pady=5)
        
        def add_and_close():
            timestamp = time_var.get()
            label = label_var.get()
            
            if timestamp <= 0 or not label:
                messagebox.showwarning("Warning", "Please enter valid time and position name")
                return
            
            self.choreo_list.insert("", "end", values=(f"{timestamp:.2f}", label))
            dialog.destroy()
        
        ttk.Button(dialog, text="Add", command=add_and_close).pack(anchor=tk.CENTER, pady=10)
        
        # Set dialog modal
        dialog.transient(self.root)
        dialog.grab_set()
        self.root.wait_window(dialog)
    
    def remove_choreography_point(self):
        """Remove selected choreography point"""
        selected = self.choreo_list.selection()
        if selected:
            self.choreo_list.delete(selected)
    
    def save_choreography(self):
        """Save the defined choreography sequence"""
        items = self.choreo_list.get_children()
        if not items:
            messagebox.showwarning("Warning", "No choreography points defined")
            return
        
        # Extract timestamps and labels
        timestamps = []
        labels = []
        
        for item in items:
            values = self.choreo_list.item(item, "values")
            timestamps.append(float(values[0]))
            labels.append(values[1])
        
        # Save to analyzer
        count = self.analyzer.define_choreography_sequence(timestamps, labels)
        
        self.status_var.set(f"Choreography sequence saved: {count} positions")
        messagebox.showinfo("Success", f"Choreography sequence with {count} positions saved successfully")
    
    def set_api_key(self):
        """Set Roboflow API key"""
        api_key = self.api_key_var.get().strip()
        if not api_key:
            messagebox.showwarning("Warning", "Please enter a valid API key")
            return
        
        # Reinitialize analyzer with API key
        self.analyzer = SpanishDanceAnalyzer(use_roboflow=True, roboflow_api_key=api_key)
        
        # Reload data
        self.analyzer.load_reference_data()
        self.analyzer.load_choreography_sequence()
        
        self.status_var.set("Roboflow API key set successfully")
        messagebox.showinfo("Success", "Roboflow API key set successfully")
    
    def load_student_video(self):
        """Load student performance video"""
        filepath = filedialog.askopenfilename(
            title="Select Student Performance Video",
            filetypes=[("Video files", "*.mp4 *.avi *.mov *.mkv")]
        )
        
        if not filepath:
            return
        
        self.student_video_path = filepath
        self.student_status_var.set(f"Selected: {os.path.basename(filepath)}")
        self.status_var.set(f"Student video selected: {os.path.basename(filepath)}")
    
    def analyze_performance(self):
        """Analyze student performance compared to demonstration"""
        if not self.demo_video_path:
            messagebox.showwarning("Warning", "Please load a demonstration video first")
            return
        
        if not self.student_video_path:
            messagebox.showwarning("Warning", "Please load a student performance video")
            return
        
        # Start progress bar
        self.analysis_progress.start()
        self.analysis_status_var.set("Analyzing performance... Please wait.")
        self.status_var.set("Analyzing student performance. This may take a while...")
        self.root.update()
        
        # Process in a separate thread
        def run_analysis():
            # Analyze performance
            results = self.analyzer.analyze_student_performance(self.student_video_path)
            
            # Update UI in main thread
            self.root.after(0, lambda: self.update_results(results))
        
        threading.Thread(target=run_analysis).start()
    
    def update_results(self, results):
        """Update the UI with analysis results"""
        # Stop progress bar
        self.analysis_progress.stop()
        
        if not results:
            self.analysis_status_var.set("Analysis failed. Please check logs.")
            self.status_var.set("Performance analysis failed")
            return
        
        self.performance_results = results
        
        # Update status
        self.analysis_status_var.set("Analysis complete")
        self.status_var.set("Performance analysis completed successfully")
        
        # Update scores in results tab
        self.overall_score_var.set(f"{results['overall_score']:.1f}%")
        self.arm_score_var.set(f"{results['arm_score']:.1f}%")
        self.foot_score_var.set(f"{results['foot_score']:.1f}%")
        
        # Clear previous results
        for i in self.results_tree.get_children():
            self.results_tree.delete(i)
        
        # Add results to treeview
        for result in results['matching_results']:
            label = result['label']
            similarity = result['overall_similarity']
            
            # Determine execution rating
            if similarity >= 90:
                execution = "Excellent"
            elif similarity >= 75:
                execution = "Good"
            elif similarity >= 60:
                execution = "Fair"
            else:
                execution = "Needs Improvement"
            
            # Generate simple feedback
            if similarity >= 90:
                feedback = f"Excellent execution of {label}"
            elif similarity >= 75:
                feedback = f"Good execution of {label} with minor adjustments needed"
            elif similarity >= 60:
                feedback = f"Fair execution of {label}, focus on form and technique"
            else:
                feedback = f"{label} needs significant improvement"
            
            self.results_tree.insert("", "end", values=(label, execution, f"{similarity:.1f}%", feedback))
        
        # Show comparison visualization
        # This would typically display the comparison frames, but we're keeping it simple here
        messagebox.showinfo("Analysis Complete", f"Performance analysis complete. Overall score: {results['overall_score']:.1f}%")
    
    def generate_report(self):
        """Generate and display detailed performance report"""
        if not self.performance_results:
            messagebox.showwarning("Warning", "No analysis results available")
            return
        
        # Generate HTML report
        html_report = self.analyzer.generate_feedback(self.performance_results)
        
        # Save HTML to temporary file
        report_path = os.path.join(self.analyzer.data_dir, "performance_report.html")
        with open(report_path, "w") as f:
            f.write(html_report)
        
        # Open in default browser
        import webbrowser
        webbrowser.open(report_path)
        
        self.status_var.set(f"Report generated and saved to {report_path}")
    
    def export_results(self):
        """Export analysis results to a file"""
        if not self.performance_results:
            messagebox.showwarning("Warning", "No analysis results available")
            return
        
        filepath = filedialog.asksaveasfilename(
            title="Save Results",
            defaultextension=".json",
            filetypes=[("JSON files", "*.json"), ("All files", "*.*")]
        )
        
        if not filepath:
            return
        
        # Create a serializable copy of results (removing non-JSON serializable objects)
        results_copy = self.performance_results.copy()
        
        # Save to JSON
        with open(filepath, "w") as f:
            json.dump(results_copy, f, indent=4)
        
        self.status_var.set(f"Results exported to {filepath}")
        messagebox.showinfo("Success", f"Results exported successfully to {filepath}")


# Main entry point
def main():
    """Main entry point of the application"""
    root = tk.Tk()
    app = SpanishDanceApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

NameError: name 'audio_classifier' is not defined

In [3]:
# MEDIAPIPE POSE ESTIMATION SETUP

import cv2
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
import os
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from IPython.display import display, HTML

# Initialize MediaPipe pose model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
    static_image_mode=True,  # Set to False for video
    model_complexity=2,      # Higher accuracy
    smooth_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

I0000 00:00:1741270696.886746 2568791 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3


In [4]:
def process_image(image_path):
    """Process an image and extract pose landmarks"""
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    return image, image_rgb, results

def visualize_pose(image_rgb, results):
    """Visualize detected pose landmarks on an image"""
    annotated_image = image_rgb.copy()
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS
    )
    plt.figure(figsize=(10, 10))
    plt.imshow(annotated_image)
    plt.title("Detected Pose")
    plt.axis('off')
    plt.show()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
#BODY PART MAPPING

# Define constants for pose comparison
POSE_LANDMARKS = {
    'left_wrist': mp_pose.PoseLandmark.LEFT_WRIST.value,
    'right_wrist': mp_pose.PoseLandmark.RIGHT_WRIST.value,
    'left_elbow': mp_pose.PoseLandmark.LEFT_ELBOW.value,
    'right_elbow': mp_pose.PoseLandmark.RIGHT_ELBOW.value,
    'left_shoulder': mp_pose.PoseLandmark.LEFT_SHOULDER.value,
    'right_shoulder': mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
    'left_hip': mp_pose.PoseLandmark.LEFT_HIP.value,
    'right_hip': mp_pose.PoseLandmark.RIGHT_HIP.value,
    'left_knee': mp_pose.PoseLandmark.LEFT_KNEE.value,
    'right_knee': mp_pose.PoseLandmark.RIGHT_KNEE.value,
    'left_ankle': mp_pose.PoseLandmark.LEFT_ANKLE.value,
    'right_ankle': mp_pose.PoseLandmark.RIGHT_ANKLE.value,
    'left_foot_index': mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value,
    'right_foot_index': mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value,
}

# Define groups for specific analysis
ARM_LANDMARKS = [
    'left_wrist', 'right_wrist', 'left_elbow', 'right_elbow', 
    'left_shoulder', 'right_shoulder'
]

FOOT_LANDMARKS = [
    'left_hip', 'right_hip', 'left_knee', 'right_knee',
    'left_ankle', 'right_ankle', 'left_foot_index', 'right_foot_index'
]

In [6]:
# REFERENCE POSE MANAGER CLASS
class ReferencePoseManager:
    """Manages reference poses for Spanish dance movements"""
    
    def __init__(self, data_dir="spanish_dance_references"):
        """Initialize the reference pose manager"""
        self.data_dir = data_dir
        self.reference_poses = {}
        self.ensure_data_dir()
        
    def ensure_data_dir(self):
        """Ensure the data directory exists"""
        if not os.path.exists(self.data_dir):
            os.makedirs(self.data_dir)
            
    def add_reference_pose(self, pose_name, image_path, pose_type="arm", description=""):
        """Add a reference pose to the database"""
        image, _, results = process_image(image_path)
        
        if not results.pose_landmarks:
            print(f"No pose detected in {image_path}")
            return False
        
        # Extract landmarks as normalized vectors
        landmarks = []
        for landmark in results.pose_landmarks.landmark:
            landmarks.append([landmark.x, landmark.y, landmark.z, landmark.visibility])
        
        self.reference_poses[pose_name] = {
            'landmarks': landmarks,
            'type': pose_type,  # 'arm' or 'foot'
            'description': description,
            'image_path': image_path
        }
        
        # Save to disk
        self.save_references()
        return True
    
    def save_references(self):
        """Save reference poses to disk"""
        with open(os.path.join(self.data_dir, "reference_poses.pkl"), "wb") as f:
            pickle.dump(self.reference_poses, f)
    
    def load_references(self):
        """Load reference poses from disk"""
        ref_file = os.path.join(self.data_dir, "reference_poses.pkl")
        if os.path.exists(ref_file):
            with open(ref_file, "rb") as f:
                self.reference_poses = pickle.load(f)
    
    def visualize_reference_pose(self, pose_name):
        """Visualize a reference pose"""
        if pose_name not in self.reference_poses:
            print(f"Reference pose '{pose_name}' not found")
            return
        
        ref_data = self.reference_poses[pose_name]
        image, image_rgb, results = process_image(ref_data['image_path'])
        
        print(f"Reference Pose: {pose_name}")
        print(f"Type: {ref_data['type']}")
        print(f"Description: {ref_data['description']}")
        
        visualize_pose(image_rgb, results)


In [7]:
# POSE ANALYZER CLASS
class PoseAnalyzer:
    """Analyzes dancer poses against reference poses"""
    
    def __init__(self, reference_manager):
        """Initialize the pose analyzer"""
        self.ref_manager = reference_manager
        
    def normalize_pose(self, landmarks):
        """Normalize pose by centering and scaling"""
        # Extract x, y coordinates
        coords = np.array([[lm.x, lm.y] for lm in landmarks])
        
        # Find the bounding box
        min_x, min_y = np.min(coords, axis=0)
        max_x, max_y = np.max(coords, axis=0)
        
        # Scale and center
        scale_x = 1.0 / (max_x - min_x) if max_x > min_x else 1.0
        scale_y = 1.0 / (max_y - min_y) if max_y > min_y else 1.0
        
        normalized = []
        for lm in landmarks:
            normalized_lm = mp_pose.PoseLandmark()
            normalized_lm.x = (lm.x - min_x) * scale_x
            normalized_lm.y = (lm.y - min_y) * scale_y
            normalized_lm.z = lm.z  # Keep z as is
            normalized_lm.visibility = lm.visibility
            normalized.append(normalized_lm)
        
        return normalized
    
    def extract_landmark_vectors(self, landmarks, landmark_group):
        """Extract vectors for specific landmark groups (arm or foot)"""
        vectors = []
        
        if landmark_group == "arm":
            target_landmarks = ARM_LANDMARKS
        elif landmark_group == "foot":
            target_landmarks = FOOT_LANDMARKS
        else:
            # Use all landmarks
            return np.array([[lm.x, lm.y, lm.z] for lm in landmarks]).flatten().reshape(1, -1)
        
        # Extract coordinates for target landmarks
        coords = []
        for name in target_landmarks:
            idx = POSE_LANDMARKS[name]
            lm = landmarks[idx]
            coords.extend([lm.x, lm.y, lm.z])
        
        return np.array(coords).reshape(1, -1)
    
    def compare_poses(self, dancer_landmarks, ref_pose_name, view="front"):
        """Compare dancer pose to a reference pose"""
        if ref_pose_name not in self.ref_manager.reference_poses:
            return {"score": 0, "message": f"Reference pose '{ref_pose_name}' not found"}
        
        ref_data = self.ref_manager.reference_poses[ref_pose_name]
        
        # Determine which landmark group to use based on view and pose type
        landmark_group = None
        if view == "front" and ref_data['type'] == 'arm':
            landmark_group = "arm"
        elif view == "side" and ref_data['type'] == 'foot':
            landmark_group = "foot"
        
        # Convert reference landmarks format
        ref_landmarks = []
        for lm_data in ref_data['landmarks']:
            lm = mp_pose.PoseLandmark()
            lm.x, lm.y, lm.z, lm.visibility = lm_data
            ref_landmarks.append(lm)
        
        # Normalize both poses
        norm_dancer = self.normalize_pose(dancer_landmarks)
        norm_ref = self.normalize_pose(ref_landmarks)
        
        # Extract vectors for comparison
        dancer_vector = self.extract_landmark_vectors(norm_dancer, landmark_group)
        ref_vector = self.extract_landmark_vectors(norm_ref, landmark_group)
        
        # Calculate similarity score (cosine similarity)
        similarity = cosine_similarity(dancer_vector, ref_vector)[0][0]
        
        # Scale to 0-100%
        score = max(0, min(100, (similarity + 1) * 50))
        
        # Generate feedback message
        message = self._generate_feedback(score, ref_pose_name, ref_data)
        
        return {
            "score": score,
            "message": message
        }
    
    def _generate_feedback(self, score, pose_name, ref_data):
        """Generate feedback based on score"""
        if score >= 90:
            return f"Excellent execution of {pose_name}! {ref_data['description']}"
        elif score >= 75:
            return f"Good execution of {pose_name}. Minor adjustments needed. {ref_data['description']}"
        elif score >= 60:
            return f"Fair execution of {pose_name}. Review the position details. {ref_data['description']}"
        else:
            return f"Needs improvement on {pose_name}. Focus on the key elements: {ref_data['description']}"
    
    def analyze_dance_sequence(self, dancer_image_path, reference_sequence, view="front"):
        """Analyze a dancer's execution of a sequence of poses"""
        dancer_image, dancer_rgb, dancer_results = process_image(dancer_image_path)
        
        if not dancer_results.pose_landmarks:
            return {"error": "No pose detected in dancer image"}
        
        # Visualize the detected pose
        visualize_pose(dancer_rgb, dancer_results)
        
        # Compare with each reference pose in the sequence
        results = {}
        for pose_name in reference_sequence:
            results[pose_name] = self.compare_poses(
                dancer_results.pose_landmarks.landmark, 
                pose_name, 
                view
            )
        
        return results


In [8]:
# DANCE EVALUATOR CLASS
class DanceEvaluator:
    """Evaluates a complete dance performance and generates reports"""
    
    def __init__(self, analyzer):
        """Initialize the dance evaluator"""
        self.analyzer = analyzer
        
    def evaluate_performance(self, dancer_frontal_image, dancer_side_image, arm_sequence, foot_sequence):
        """Evaluate a dance performance from both frontal and side views"""
        # Analyze arm movements (frontal view)
        print("Analyzing arm movements (frontal view)...")
        arm_results = self.analyzer.analyze_dance_sequence(
            dancer_frontal_image, 
            arm_sequence, 
            view="front"
        )
        
        # Analyze footwork (side view)
        print("Analyzing footwork (side view)...")
        foot_results = self.analyzer.analyze_dance_sequence(
            dancer_side_image, 
            foot_sequence, 
            view="side"
        )
        
        # Combine results
        complete_evaluation = {
            "arm_movements": arm_results,
            "footwork": foot_results
        }
        
        return complete_evaluation
    
    def generate_feedback_report(self, evaluation_results):
        """Generate a comprehensive feedback report"""
        arm_results = evaluation_results.get("arm_movements", {})
        foot_results = evaluation_results.get("footwork", {})
        
        # Calculate overall performance score
        all_scores = []
        for pose_name, result in arm_results.items():
            if isinstance(result, dict) and "score" in result:
                all_scores.append(result["score"])
        
        for pose_name, result in foot_results.items():
            if isinstance(result, dict) and "score" in result:
                all_scores.append(result["score"])
        
        overall_score = np.mean(all_scores) if all_scores else 0
        
        # Generate HTML report
        html = f"""
        <h2>Spanish Dance Performance Evaluation</h2>
        <h3>Overall Score: {overall_score:.1f}%</h3>
        
        <h3>Arm Movements (Frontal View)</h3>
        <table border="1" style="border-collapse: collapse; width: 100%;">
        <tr>
            <th style="padding: 8px; text-align: left;">Position</th>
            <th style="padding: 8px; text-align: left;">Score</th>
            <th style="padding: 8px; text-align: left;">Feedback</th>
        </tr>
        """
        
        # Add arm movement results
        for pose_name, result in arm_results.items():
            if isinstance(result, dict) and "score" in result:
                score = result["score"]
                message = result.get("message", "")
                
                # Determine rating color based on score
                if score >= 90:
                    color = "green"
                    rating = "Excellent"
                elif score >= 75:
                    color = "#5cb85c"  # lighter green
                    rating = "Good"
                elif score >= 60:
                    color = "orange"
                    rating = "Fair"
                else:
                    color = "red"
                    rating = "Needs Improvement"
                
                html += f"""
                <tr>
                    <td style="padding: 8px;">{pose_name}</td>
                    <td style="padding: 8px; color: {color};">{score:.1f}% ({rating})</td>
                    <td style="padding: 8px;">{message}</td>
                </tr>
                """
        
        html += """
        </table>
        
        <h3>Footwork (Side View)</h3>
        <table border="1" style="border-collapse: collapse; width: 100%;">
        <tr>
            <th style="padding: 8px; text-align: left;">Position</th>
            <th style="padding: 8px; text-align: left;">Score</th>
            <th style="padding: 8px; text-align: left;">Feedback</th>
        </tr>
        """
        
        # Add footwork results
        for pose_name, result in foot_results.items():
            if isinstance(result, dict) and "score" in result:
                score = result["score"]
                message = result.get("message", "")
                
                # Determine rating color based on score
                if score >= 90:
                    color = "green"
                    rating = "Excellent"
                elif score >= 75:
                    color = "#5cb85c"  # lighter green
                    rating = "Good"
                elif score >= 60:
                    color = "orange"
                    rating = "Fair"
                else:
                    color = "red"
                    rating = "Needs Improvement"
                
                html += f"""
                <tr>
                    <td style="padding: 8px;">{pose_name}</td>
                    <td style="padding: 8px; color: {color};">{score:.1f}% ({rating})</td>
                    <td style="padding: 8px;">{message}</td>
                </tr>
                """
        
        html += """
        </table>
        
        <h3>Areas for Improvement</h3>
        <p>Focus on the following positions:</p>
        <ul>
        """
        
        # Find lowest-scoring positions for improvement focus
        all_results = []
        for pose_name, result in arm_results.items():
            if isinstance(result, dict) and "score" in result:
                all_results.append((pose_name, result["score"], "arm"))
        
        for pose_name, result in foot_results.items():
            if isinstance(result, dict) and "score" in result:
                all_results.append((pose_name, result["score"], "foot"))
        
        # Sort by score (ascending) and take the 3 lowest
        all_results.sort(key=lambda x: x[1])
        for pose_name, score, pose_type in all_results[:3]:
            html += f"<li><strong>{pose_name}</strong> ({pose_type} position): {score:.1f}%</li>"
        
        html += """
        </ul>
        
        <p>Practice these positions to improve your overall performance.</p>
        """
        
        # Display the HTML report
        display(HTML(html))
        
        return html
    
    def display_report(self, evaluation_results):
        """Display the feedback report"""
        return self.generate_feedback_report(evaluation_results)


In [9]:
# EXAMPLE USAGE
# Create the reference pose manager and add sample reference poses
def setup_reference_dataset():
    """Set up a sample reference dataset"""
    ref_manager = ReferencePoseManager(data_dir="spanish_dance_references")
    
    # Example: Add reference poses for arm movements (frontal view)
    arm_poses = {
        "brazos_en_quinta": {
            "path": "dataset/arms/quinta.jpg",
            "description": "Arms in fifth position, rounded above the head"
        },
        "braceo_flamenco": {
            "path": "dataset/arms/braceo.jpg",
            "description": "Flowing arm movement with wrists rotating"
        }
    }
    
    for name, data in arm_poses.items():
        ref_manager.add_reference_pose(name, data["path"], "arm", data["description"])
    
    # Example: Add reference poses for footwork (side view)
    foot_poses = {
        "planta_tacón": {
            "path": "dataset/feet/planta_tacon.jpg",
            "description": "Heel-to-toe rhythmic step"
        },
        "golpe": {
            "path": "dataset/feet/golpe.jpg",
            "description": "Sharp stamp of the whole foot"
        }
    }
    
    for name, data in foot_poses.items():
        ref_manager.add_reference_pose(name, data["path"], "foot", data["description"])
    
    return ref_manager

# Cell 8: Evaluate a dancer's performance
def evaluate_dancer_performance(dancer_name, frontal_image_path, side_image_path):
    """Evaluate a dancer's performance"""
    # Load the reference dataset
    ref_manager = ReferencePoseManager(data_dir="spanish_dance_references")
    ref_manager.load_references()
    
    # Create the analyzer and evaluator
    analyzer = PoseAnalyzer(ref_manager)
    evaluator = DanceEvaluator(analyzer)
    
    # Define the sequence of positions to evaluate
    arm_sequence = ["brazos_en_quinta", "braceo_flamenco"]
    foot_sequence = ["planta_tacón", "golpe"]
    
    # Evaluate the dancer's performance
    results = evaluator.evaluate_performance(
        frontal_image_path,
        side_image_path,
        arm_sequence,
        foot_sequence
    )
    
    # Display the evaluation report
    print(f"Performance Evaluation for {dancer_name}")
    evaluator.display_report(results)
    
    return results

# Cell 9: Run evaluation on test data
# Uncomment and run when ready
# setup_reference_dataset()
# evaluate_dancer_performance(
#     "Test Dancer",
#     "test_data/dancer_front.jpg",
#     "test_data/dancer_side.jpg"
# )


In [ ]:
#SPANISH DANCE POSE ANALYSIS TOOL

import cv2
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# GUI setup
root = tk.Tk()
root.title("Spanish Dance Pose Analysis Tool")
root.geometry("800x600")

# Function to load predefined images
def load_predefined_images():
    global predefined_images
    file_paths = filedialog.askopenfilenames(title="Select Predefined Position Images", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    predefined_images = [cv2.imread(file) for file in file_paths]
    messagebox.showinfo("Info", f"Loaded {len(predefined_images)} predefined images")

# Function to load exercise video
def load_exercise_video():
    global exercise_video_path
    exercise_video_path = filedialog.askopenfilename(title="Select Exercise Video", filetypes=[("Video Files", "*.mp4;*.avi;*.mov")])
    messagebox.showinfo("Info", "Exercise video loaded successfully")

# Function to start real-time camera observation
def start_camera():
    cap = cv2.VideoCapture(0)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert image to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process frame using MediaPipe Pose
        results = pose.process(rgb_frame)
        
        if results.pose_landmarks:
            for landmark in results.pose_landmarks.landmark:
                h, w, _ = frame.shape
                x, y = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
        
        cv2.imshow("Real-Time Pose Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Function to generate checklist and report
def generate_report():
    # Placeholder logic for evaluation
    messagebox.showinfo("Report", "Checklist and report generated successfully!")

# GUI Buttons
btn_load_images = tk.Button(root, text="Load Predefined Images", command=load_predefined_images)
btn_load_images.pack(pady=10)

btn_load_video = tk.Button(root, text="Load Exercise Video", command=load_exercise_video)
btn_load_video.pack(pady=10)

btn_start_camera = tk.Button(root, text="Start Camera Observation", command=start_camera)
btn_start_camera.pack(pady=10)

btn_generate_report = tk.Button(root, text="Generate Report", command=generate_report)
btn_generate_report.pack(pady=10)

root.mainloop()

W0000 00:00:1741270696.943663 2573922 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1741270696.946025 2568791 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3
W0000 00:00:1741270696.984349 2573921 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741270697.000711 2573940 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741270697.015037 2573939 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-03-06 15:18:53.383 python[44073:2568791] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-06 15:18:53.383 python[44073:2568791] +[IMKInputSession subc